In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def fast_thinning(img):
    skel = np.zeros(img.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    while True:
        open = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
        temp = img - open
        eroded = cv2.erode(img, element)
        skel = cv2.bitwise_or(skel, temp)
        img = eroded.copy()
        if cv2.countNonZero(img) == 0:
            break
    return skel

In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar la imagen
img_path = 'testing/inference_dem_2/class_epoch_99.png'
img = cv2.imread(img_path)
img = img[:,:,::-1]
img_red = img[:, :, 0]
img_green = img[:, :, 1]
img_blue = img[:, :, 2]

# Crear una máscara binaria para detectar píxeles con color verde puro
green_mask = np.logical_and(img_green > img_red, img_green > img_blue)
binary_img = green_mask.astype(np.uint8) * 255

# Operaciones morfológicas optimizadas
binary_img = cv2.medianBlur(binary_img, 3)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
binary_img = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernel, iterations=2)
binary_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel, iterations=1)

# Análisis de conectividad con filtrado temprano
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_img, connectivity=8)
valid_components = [i for i in range(1, num_labels) if stats[i, cv2.CC_STAT_AREA] > 150]

# Pre-alocar arrays
colored_labels = np.zeros_like(img)
skeleton_img = np.zeros(img_green.shape, dtype=np.float32)
contours_img = np.zeros_like(img_green)
largest_contours = []

for i in valid_components:
    mask = (labels == i)
    colored_labels[mask] = [np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256)]
    
    
    mask = mask.astype(np.uint8)
    x, y, w, h = cv2.boundingRect(mask)
    roi = mask[y:y+h, x:x+w]
    skeleton = fast_thinning(roi)
    skeleton_img[y:y+h, x:x+w] += skeleton
    
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        largest_contour += (x, y)  # Ajustar la posición del contorno
        largest_contours.append(largest_contour)

for contour in largest_contours:
    cv2.drawContours(contours_img, [contour], -1, 255, 2)

skeleton_img = np.clip(skeleton_img,0,1)
c = np.stack((contours_img, skeleton_img*255, np.zeros_like(skeleton_img)), axis=-1)

plt.figure(figsize=(13, 5))
plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.imshow(binary_img, cmap='gray')
plt.subplot(1, 4, 2)
plt.title('Components')
plt.imshow(colored_labels)
plt.subplot(1, 4, 3)
plt.title('Skeleton')
plt.imshow(skeleton_img, cmap='grey')
plt.subplot(1, 4, 4)
plt.title('Skeleton + Contours')
plt.imshow(c)
plt.show()


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..255.0].
